<a href="https://colab.research.google.com/github/sarannetworkprogammer/Data_foundations/blob/main/Saran_PS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 3

For this problem set, you will expand on PS2 to perform and evaluate various sentiment classification methods.

Your name: 

You abc123:

## Submission Instructions

After completing the exercises below, generate a pdf of the code **with** outputs. After that create a zip file containing both the completed exercise and the generated PDF/HTML. You are **required** to check the PDF/HTML to make sure all the code **and** outputs are clearly visible and easy to read. If your code goes off the page, you should reduce the line size. I generally recommend not going over 80 characters.

Finally, name the zip file using a combination of your the assigment and your name, e.g., ps3_rios.zip

## Exercise 1 (1 point)

For this step, you will load the training and test sentiment datasets "twitdata_TEST.tsv" and "allTrainingData.tsv". The data should be loaded into 4 lists of strings: X_txt_train, X_txt_test, y_test, y_train.

Note, when using csvreader, you need to pass the "quoting" the value csv.QUOTE_NONE.

In [2]:
import csv
X_txt_train = []
y_train = []

# Loading data from CSVs.


with open("/content/drive/MyDrive/Colab Notebooks/PS3/allTrainingData.tsv") as file:
 
   tsv_file = csv.reader(file, delimiter="\t",quoting=csv.QUOTE_NONE)
   for line in tsv_file:
     X_txt_train.append(line[3])
     y_train.append(line[2])


print(len(X_txt_train))
print(len(y_train))
"""
# 1. Load the training datasets into two lists (X_txt_train will be a list of strings; y_train)

X_txt_test = ...
y_test = ...
# 2. Load the test datasets into two lists (X_txt_test will be a list of strings; y_test)
"""
X_txt_test = []
y_test = []


with open("/content/drive/MyDrive/Colab Notebooks/PS3/twitdata_TEST.tsv") as in_file:

  reader = csv.reader(in_file, delimiter="\t",quoting=csv.QUOTE_NONE)
  count = 0
  for row in reader:
    X_txt_test.append(row[3])
    y_test.append(row[2])

print("test data")
print(len(X_txt_test))
print(len(y_test))

8018
8018
test data
3199
3199


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [3]:
assert(type(X_txt_train) == type(list()))
assert(type(X_txt_train[0]) == type(str()))
assert(type(X_txt_test) == type(list()))
assert(type(X_txt_test[0]) == type(str()))
assert(type(y_test) == type(list()))
assert(type(y_train) == type(list()))
assert(len(X_txt_test) == 3199)
assert(len(y_test) == 3199)
assert(len(X_txt_train) == 8018)
assert(len(y_train) == 8018)
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 2 (2 point)

This part is similar to HW2 (using the positive_words and negative_words variables). We will compare last homework's lexicon-based classification method with supervised models. Only make predictions on the test split and store all predictions in the list lex_test_preds. Next, calculate the **macro** precision, macro recall, and macro f1 scores using the lex_test_preds list.

You can learn more about lexicon-based classification in Chapter 19.6. If you are interested, the chapter is available online for free at the following link: [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/19.pdf)

Also, note that I have wrote the code for this exercise, you job is to read the code, understand it, and apply it to make the predictions.

**INTUITION:** For PS2 you implemented a "lexicon-based classifier". You looked at a few examples and manually accessed it's performance. Howeover, that was arbitary. Now we want to see how well it actually works. Hence, in this homework (for this exercise), you will use the class I provided that implements the lexicon-based classifier and use the provided "annotatoed dataset" (loaded in Exercise 1) to see how well it performance. If it worked 100\% accurately, the F1 would be 1.00. However, it does not, so you should expect a smaller score.

In [4]:
# DO NOT MODIFY THE CODE IN THIS CELL
class LexiconClassifier():
    def __init__(self):
        """
            Initalize the Lexicon classifer by loading lexicons. 
        """
        self.positive_words = set()
        with open('/content/drive/MyDrive/Colab Notebooks/PS3/positive-words.txt', encoding = 'utf-8') as iFile:
            for row in iFile:
                self.positive_words.add(row.strip())

        self.negative_words = set()
        with open('/content/drive/MyDrive/Colab Notebooks/PS3/negative-words.txt', encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.negative_words.add(row.strip())

    def predict(self, sentence):
        """
            Returns a sentiment prediction give an input string.
            
            Keyword arguments:
            sentence -- string (e.g., "This is good good good")
            
            Returns:
            pred -- a string ("postive, "negative", or "neutral")
        """
        num_pos_words = 0
        num_neg_words = 0
        for word in sentence.lower().split():
            if word in self.positive_words:
                num_pos_words += 1
            elif word in self.negative_words:
                num_neg_words += 1
        
        pred = 'neutral'        
        if num_pos_words > num_neg_words:
            pred = 'positive'
        elif num_pos_words < num_neg_words:
            pred = 'negative'
            
        return pred
    
    def count_pos_words(self, sentence):
        """
            Returns the number of positive words in string
            
            Keyword arguments:
            sentence -- string (e.g., "This is good good good")
            
            Returns:
            pred -- an integer (e.g., 3)
        """
        num_pos_words = 0
        for word in sentence.lower().split():
            if word in self.positive_words:
                num_pos_words += 1
        return num_pos_words

    def count_neg_words(self, sentence):
        """
            Returns the number of negative words in string
            
            Keyword arguments:
            sentence -- string (e.g., "This is good good good")
            
            Returns:
            pred -- an integer (e.g., 3)
        """
        num_neg_words = 0
        for word in sentence.lower().split():
            if word in self.negative_words:
                num_neg_words += 1
        return num_neg_words

In [5]:
# WRITE CODE HERE
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. Instatiate that class

lexicon_classifier = LexiconClassifier()

lex_test_preds = [] # Initialize this as an empty list

# Loop over X_txt_test
#    for each string in X_txt_test (i.e., for each item in the list), pass it to LexiconClassifiers .predict() method
#    append the prediction to lex_test_preds
count = 0
for each in X_txt_test:

  prediction = lexicon_classifier.predict(each)
  lex_test_preds.append(prediction)
  #print(each)
  #count = count + 1
  #if count == 2:
    #break


print(len(lex_test_preds))

precision = precision_score(y_test, lex_test_preds, average='macro')            # Get scores using lex_test_preds and y_test with the precision_score method
recall =  recall_score(y_test, lex_test_preds, average='macro')                 # Get scores using lex_test_preds and y_test with the recall_score method
f1 = f1_score(y_test, lex_test_preds, average='macro')                          # Get scores using lex_test_preds and y_test with the f1_score method



print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))


3199
Precision: 0.5504
Recall: 0.5446
F1: 0.5455


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [6]:
assert(type(lex_test_preds) == type(list()))
assert(type(lex_test_preds[0]) == type(str()))
assert(set(lex_test_preds) == set(["positive", "negative", "neutral"]))
assert(len(lex_test_preds) == len(y_test))
assert(type(precision) == type(float()) or type(precision) == type(np.float64()))
assert(type(recall) == type(float()) or type(recall) == type(np.float64()))
assert(type(f1) == type(float()) or type(f1) == type(np.float64()))
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 3 (1 point)

Again, using the LexiconClassifier, write code to generate a lists of lists where each sublist contains the number of positive words and negative words in a tweet. For example, assume we are given the following train and test datasets

``` python
X_txt_train = ["good good", "bad bad"]
X_txt_test = ["great", "bad bad great"]
```

you should write code that creates two lists of lists as follows:

``` python
X_train_lexicon_features = [[2, 0], [0,2]] # [2, 0] means the first tweeta has 2 positive words and 0 negative words.
X_test_lexicon_features = [[1, 0], [1, 2]]
```

Why are we doing this? We will use these as addition features in Exercise 5, combining it with the ngram features. Combining different sets of features is called "Feature Engineering" and is one of the most important steps of many machine learning tasks. In this case, we are using the lexicons to generate additional features. But, we could also count the number of capitalized words, number of punctuation marks, etc. We would come up with different feature sets via trial-and-error. We can guess what type of features would help our task. For instance, for sentiment prediction, we may guess that having many capitalized words is predictive of something negative (e.g., "WHY ARE YOU DOING THIS!!!!!").



In [7]:
# WRITE CODE HERE

X_train_lexicon_features = [] # Initailze to an empty list. This will be a list of lists
X_test_lexicon_features = [] #  Initailze to an empty list. This will be a list of lists

# Loop over X_txt_test
#    for each string in X_txt_test (i.e., for each item in the list), pass it to LexiconClassifiers .count_pos_words() and count_neg_words method
#    append a list with the counts to X_test_lexicon_features

test =[]
for each in X_txt_test:

  count = count + 1
  
  pos= lexicon_classifier.count_pos_words(each)
  neg = lexicon_classifier.count_neg_words(each)
  X_test_lexicon_features.append([pos,neg])



# Loop over X_txt_train
#    for each string in X_txt_train (i.e., for each item in the list), pass it to LexiconClassifiers .count_pos_words() and count_neg_words method
#    append a list with the counts to X_train_lexicon_features


for each in X_txt_train:

  count = count + 1
  
  pos1= lexicon_classifier.count_pos_words(each)
  neg1 = lexicon_classifier.count_neg_words(each)
  X_train_lexicon_features.append([pos1,neg1])


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [8]:
assert(type(X_train_lexicon_features) == type(list()))
assert(type(X_test_lexicon_features) == type(list()))
assert(type(X_test_lexicon_features[0]) == type(list()))
assert(len(X_train_lexicon_features) == len(X_txt_train))
assert(len(X_test_lexicon_features) == len(X_txt_test))
assert(len(X_train_lexicon_features[0]) == 2)
assert(len(X_test_lexicon_features[0]) == 2)
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 4 (2 points)

For this task you should creat a feature matrix using CountVectorizer and train a LinearSVC model from scikit-learn. On the train split, use GridSearchCV to find the best LinearSVC C values (0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10, or 100) based on the **macro** f1 scoring metric (hint: "macro" average) and set the cv parameter to 5. Also, with the CountVectorizer, only use unigrams (i.e., set ngram_range = (1,1)). Note that GridSearchCV will retrain the final classifier using the best parameters, so you don't need to do it manually.

**INTUITION:** For this exercise, you are implementing a simple linear model using bag-of-words features. This is generally a very strong and simple baseline for text classification. Compare the scores from this exercise to the results in Exercise 2. You will find that the machine learning-based model implemented here acheives better performance.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score

import numpy as np
np.random.seed(42)
import random
random.seed(42)

### warning disabling

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


# WRITE CODE HERE

# Summary:
# 1. Convert X_txt_train and X_txt_test to matricies of numbers (i.e., use CountVectorizer)

vec = CountVectorizer(ngram_range=(1,1))

X_train = vec.fit_transform(X_txt_train) # This should be a matrix
X_test = vec.transform(X_txt_test)   # This should be a matrix


print("shapes")

print(X_train.shape)
print(X_test.shape)
# Initialize the classifier LinearSVC 

model = LinearSVC()

# Create the params with the C values

params = {"C": [0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10,100]}

# Initialize GridSearchCV

clf = GridSearchCV(model, param_grid=params, cv=5, scoring='f1_macro')

# "fit" the model  on X_train

clf.fit(X_train, y_train)

validation_score = clf.best_score_  # Get the score from the GridSearchCV "best score"

print("Validation F1: {:.4f}".format(validation_score))

svm_test_predictions =  clf.predict(X_test) # "predict" on X_test 

precision = precision_score(y_test, svm_test_predictions, average='macro')  # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_test, svm_test_predictions, average='macro')
f1 = f1_score(y_test, svm_test_predictions, average='macro')
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))




shapes
(8018, 20864)
(3199, 20864)
Validation F1: 0.5921
Precision: 0.6525
Recall: 0.5740
F1: 0.5878


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [10]:
from scipy.sparse import csr_matrix
assert(type(X_train) == type(csr_matrix(0)) or type(X_train) == type(np.array(0)))
assert(type(X_test) == type(csr_matrix(0)) or type(X_test) == type(np.array(0)))
assert(X_train.shape[0] == len(X_txt_train))
assert(X_test.shape[0] == len(X_txt_test))
assert(X_train.shape[1] == X_test.shape[1])
assert(type(precision) == type(float()) or type(precision) == type(np.float64()))
assert(type(recall) == type(float()) or type(recall) == type(np.float64()))
assert(type(f1) == type(float()) or type(f1) == type(np.float64()))
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 5 (2 points)

Repeat the experiment from exercise 4, but include the lexicon features (from exercise 3) with the CountVectorizer features. Specifically, you need to concatenate the variables ```X_train_lexicon_features``` and ```X_test_lexicon_features``` with ```X_train``` and ```X_test```, respectively. Intuitively, we are performing feature engineering by adding "lexicon features".

HINT: You will need to convert the lexicon features to numpy arrays then call hstack from the scipy.sparse library (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.hstack.html)

```python
    from scipy.sparse import hstack
    new_matrix = hstack([ngram_matrix, lexicon_matrix])
```

Again, this is a "Feature Engineering" exercise. This is common in all machine learning tasks.

**INTUITION:** How do we improve the model Exercise 4? You could try different machine learning models. However, it is generally better to focus on feature engineering. What information can you provide the model to make better predictions? Here we will try to combine counts using the Lexicon from Exercise 3 as additional features (i.e., combined with the bag-of-word features) from Exercise 4.

In [25]:
import scipy.sparse as sp
from scipy.sparse import hstack
import numpy as np
np.random.seed(42)
import random
random.seed(42)

# WRITE CODE HERE


# Summary:
# 1. Convert X_txt_train and X_txt_test to matricies of numbers (i.e., use CountVectorizer)

vec2 = CountVectorizer(ngram_range=(1,1))

X_train_w_lex = vec2.fit_transform(X_txt_train) # This will be the matrix from CountVectorizer (X_txt_train)
X_test_w_lex = vec2.transform(X_txt_test)

# Now we need to convert X_train_lexicon_features and X_test_lexicon_features to numpy arrays
X_train_lexicon_features = np.array(X_train_lexicon_features)
X_test_lexicon_features = np.array(X_test_lexicon_features)

# "hstack" X_train_lexicon_features with X_train_w_lex
# "hstack" X_test_lexicon_features with X_test_w_lex

X_train_w_lex =  hstack([X_train_lexicon_features, X_train_w_lex])
X_test_w_lex  = hstack([X_test_lexicon_features,X_test_w_lex])

# Initialize the classifier LinearSVC 

model = LinearSVC()

# Create the params with the C values

params = {"C": [0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10,100]}

# Initialize GridSearchCV

clf = GridSearchCV(model, param_grid=params, cv=4, scoring='f1_macro')

# "fit" the model  on X_train_w_lex

clf.fit(X_train_w_lex,y_train)

validation_score = clf.best_score_ 
print("Validation F1: {:.4f}".format(validation_score))

svm_lex_test_predictions =  clf.predict(X_test_w_lex )  # Get predictions on X_test_w_lex

precision = precision_score(y_test, svm_lex_test_predictions , average='macro')  # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_test, svm_lex_test_predictions, average='macro')
f1 = f1_score(y_test, svm_lex_test_predictions, average='macro')
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))

Validation F1: 0.6008
Precision: 0.6617
Recall: 0.5977
F1: 0.6124


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [18]:
from scipy.sparse import csr_matrix
assert(X_train_w_lex.shape[0] == len(X_txt_train))
assert(X_test.shape[0] == len(X_txt_test))
assert(X_train_w_lex.shape[1] == X_test.shape[1] + 2)
assert(X_train_w_lex.shape[1] == X_test_w_lex.shape[1])
assert(type(precision) == type(float()) or type(precision) == type(np.float64()))
assert(type(recall) == type(float()) or type(recall) == type(np.float64()))
assert(type(f1) == type(float()) or type(f1) == type(np.float64()))
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 6 (1 point)

For this exercise, you will perform manual analysis of the predictions. Answer the questions below.

In [19]:
num_tweets = 0
for text, svm_pred, svm_lex_pred, lex_pred, y  in zip(X_txt_test, svm_test_predictions, svm_lex_test_predictions, lex_test_preds, y_test):
    print("Tweet: {}".format(text))
    print("Ground-Truth Class: {}".format(y))
    print("SVM Prediction: {}".format(svm_pred))
    print("SVM+Lexicon Prediction: {}".format(svm_lex_pred))
    print("Lexicon Model Prediction: {}".format(lex_pred))
    print()
    
    num_tweets += 1
    if num_tweets == 20:
        break

Tweet: Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not
Ground-Truth Class: positive
SVM Prediction: positive
SVM+Lexicon Prediction: positive
Lexicon Model Prediction: positive

Tweet: On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6
Ground-Truth Class: neutral
SVM Prediction: neutral
SVM+Lexicon Prediction: neutral
Lexicon Model Prediction: negative

Tweet: Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt
Ground-Truth Class: negative
SVM Prediction: neutral
SVM+Lexicon Prediction: neutral
Lexicon Model Prediction: positive

Tweet: Tomorrow come and hear @DavidWillettsMP&amp;@MASieghart debate "Navigating the new Higher Education market" 5.30pm, Jurys Inn #CPC12
Ground-Truth Class: neutral
SVM Prediction: neutral
SVM+Lexicon Prediction: neutral
Lexicon Model Predict

Manually annotate all of the tweets printed above:
   1. Tweet 1 Annotation Here: Positive
   1. Tweet 2 Annotation Here: Negative
   1. Tweet 3 Annotation Here: Neutral
   1. Tweet 4 Annotation Here: Negative
   1. Tweet 5 Annotation Here: Negative
   1. Tweet 6 Annotation Here: Neutral
   1. Tweet 7 Annotation Here: Neutral
   1. Tweet 8 Annotation Here: Neutral
   1. Tweet 9 Annotation Here: Positive
   1. Tweet 10 Annotation Here: Positive
   1. Tweet 11 Annotation Here: Neutral
   1. Tweet 12 Annotation Here: Neutral
   1. Tweet 13 Annotation Here: Positive
   1. Tweet 14 Annotation Here: Positive
   1. Tweet 15 Annotation Here: Positive
   1. Tweet 16 Annotation Here: Positive
   1. Tweet 17 Annotation Here: Neutral
   1. Tweet 18 Annotation Here: Neutral
   1. Tweet 19 Annotation Here: Positive
   1. Tweet 20 Annotation Here: Positive

- How many of your annotations match the ground truth labels? Do you think the datasets labels are correct? (Use your intuition)
    - Most of them are matched except three. Yeah i think dataset labels are correct.

- How many of your annotations match the lexicon-based model's predictions?
    - 12 of my annotations mathced with lexicon based models's predictions.

- How many of your annotations match the SVM's predictions?
    - 13 of my annotations matchedd with my SVM's predictions.
    
- How many of your annotations match the SVM+Lexicon's predictions?
    - 14 of my annotations matched with SVM+Lexicon predictions.
    
- Do you see any major limitations of the linear SVM model? Use your intuition, I will accept most answers, as long as it makes some sense. Please describe and provide examples below:


Disadvantages:
SVM algorithm is not suitable for large data sets.
SVM does not perform very well when the data set has more noise i.e. target classes are overlapping.
In cases where the number of features for each data point exceeds the number of training data samples, the SVM will underperform.
As the support vector classifier works by putting data points, above and below the classifying hyperplane there is no probabilistic explanation for the classification.

## Exercise 6 (1 point)

For this exercise, you should come up with 10 other potential features that could be useful for sentiment analysis. You do not need to implement them. You simply need to list this. Make sure it is easy for me to understand the feature you describe. An example could be "The count of the number of capitalized words in the text".

1. Count number of captalized words
2. Counting all positive words
3. coutning negative words
4. score of postive and negative word
5. checking not before positive words
6. whether it is opinion or suggestion
7. Neutral statements
8. factual or personal feeling
9. Multilingual sentiment analysis
10. aspect-based

## Extra Credit 1 (2 points)

For this extra credit the only goal is to improve your model on the test set (i.e., increase the **macro** f1 score). You may create new features, grid search over more parameters, try different feature weighting methods (e.g., TfidfVectorizer), or test different machine learning models. You can do whatever you want as long as the final test score improves, I will provide you with the extra credit points.

DO NOT TRAIN ON THE TEST SET. That is cheating!

In [22]:
# WRITE CODE HERE
# Here Im using tf-idf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer


vec = TfidfVectorizer(analyzer='word' , stop_words='english',)

X_train = vec.fit_transform(X_txt_train) # This should be a matrix
X_test = vec.transform(X_txt_test)   # This should be a matrix

print(X_train.shape)
print(X_test.shape)

model = LinearSVC()

# Create the params with the C values

params = {"C": [0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10,100]}

# Initialize GridSearchCV

clf = GridSearchCV(model, param_grid=params, cv=5, scoring='f1_macro')

# "fit" the model  on X_train

clf.fit(X_train, y_train)

validation_score = clf.best_score_  # Get the score from the GridSearchCV "best score"

print("Validation F1: {:.4f}".format(validation_score))

svm_test_predictions =  clf.predict(X_test) # "predict" on X_test 

precision = precision_score(y_test, svm_test_predictions, average='macro')  # Get scores using svm_test_predictions and y_test with the precision_score method
recall = recall_score(y_test, svm_test_predictions, average='macro')
f1 = f1_score(y_test, svm_test_predictions, average='macro')
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))



(8018, 20586)
(3199, 20586)
Validation F1: 0.5762
Precision: 0.6423
Recall: 0.5658
F1: 0.5788
